<a href="https://colab.research.google.com/github/markumreed/colab_sklearn/blob/main/logistic_regression_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression with Python

- Predict a classification- survival or deceased. 
- Implemente Logistic Regression in Python for classification.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
titanic = sns.load_dataset("titanic")